In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, FloatType, DoubleType
import os

In [0]:
%run ../Utils/DBInstanceUtils

In [0]:
%run ../Utils/ProcessingUtils

In [0]:
%run ../Utils/FormatterUtils

In [0]:
%run ../Utils/FileUtils

In [0]:
instance = DBInstanceUtils('treinamento_formula1')
file1_name = 'qualifying_split_1'
file2_name = 'qualifying_split_2'
extension = 'json'

file1 = f'{file1_name}.{extension}'
file2 = f'{file2_name }.{extension}'

abs_path = f'{instance.get_filepath()}/qualifying/'.replace('file:','')

schema = StructType(
    [
        StructField("constructorId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("number", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("q1", StringType(), True),
        StructField("q2", StringType(), True),
        StructField("q3", StringType(), True),
        StructField("qualifyId", IntegerType(), True),
        StructField("raceId", IntegerType(), True)
    ]
)

In [0]:
try:
    df = (spark.read
        .format('json')
        .option('mode', 'FAILFAST')
        .schema(schema)
        .load(f'{instance.get_filepath()}/qualifying/{file1}'))
except Exception:
    print('CORRUPTED RECORDS DETECTED, ATTEMPTING TO FIX...')
    try:
      FileUtils.fix_corrupted_file(abs_path, file1, 'json')
    except Exception as fatal_error:
        dbutils.notebook.exit(f'READ FAILED!! - ABORTING PROGRAM\n{fatal_error}')

In [0]:
df = (Refine(df)
      .camelCase_to_snake_case()
      .load())

df.display()

In [0]:
#filename    = 'pitstopsRaw'
#extension   = 'delta'
#file        = filename + '.' + extension
#DBInstanceUtils.set_dbfs_savepath(instance, 'mnt', f'formula1/bronze/pitstops', file)

In [0]:
#df.write.format("delta").mode("overwrite").save(instance.get_dbfs_savepath())